# Assignment 4

Code from Assignment 3 for tweet sentiment analysis data

In [150]:
#sentiment Analysis Data

import tweepy
import csv
import pandas as pd

#using twitter Authentication to connect to twitter api with twitter provided keys.
authentication = tweepy.OAuthHandler('Dpp7E4W9YkKGVszqezpjpSlLc', 'bczWmE17Ve1DJHKJY0Z9eQYuAupPZ63JfL9AGnkuX12SUZkCWJ')
authentication.set_access_token('4289066089-Vd6LpZcoLUERdoFJTnk3J12w8M2sgajQ8HDIlaL', 'yebOZX98i8RtrsMNcWrLBh54K6ypZXi3KejgXSzunyxnH')
api = tweepy.API(authentication,wait_on_rate_limit=True)

#file operation
csvFile = open('tagdata.csv', 'w')
data = pd.read_csv('tagdata.csv', names=['tweet_data'], header=None)
write_to_csv = csv.writer(csvFile)

#writing to file using csvwriter
for tweet_data in tweepy.Cursor(api.search,q="#wuhancoronavirus",lang="en").items(100):
  write_to_csv.writerow([tweet_data.text.encode('utf-8')])
#reading csv file 
data = pd.read_csv('tagdata.csv',error_bad_lines=False,names=['tweet_data'])
import string

#Removing noise (Punctuation,extra words)
data['tweet_data']=data['tweet_data'].str.replace('[{}]'.format(string.punctuation), '')
data['tweet_data']=data['tweet_data'].str.replace('[^A-Za-z ]','')
data['tweet_data']=data['tweet_data'].str.replace('bRT','')
# print(data['tweet_data'].head())

#Removing numbers
data['tweet_data'] = data['tweet_data'].str.replace('\d+', '')

#Removing StopWords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')
data['tweet_data']=data['tweet_data'].apply(lambda x:" ".join(y for y in x.split() if y not in stop))


#Lower casing
data['tweet_data']=data['tweet_data'].str.lower()
data2=data["tweet_data"]
#stemming
from nltk.stem import PorterStemmer
st=PorterStemmer()
data['tweet_data']=data['tweet_data'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmetization
from textblob import Word
import nltk
nltk.download('wordnet')
data['tweet_data']=data['tweet_data'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['tweet_data'].head(5)
#saving the formatted sentences to "Formatted_sentences.txt"
# data['tweet_data'].to_csv("Formatted_tweets.csv",header="Tweets",index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\satya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\satya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    desertfoxi thi rich nnfirst unleash wuhancoron...
1    jack a number satir poster featur xi jin ping ...
2    bin meantim peopl die lack access medic treatm...
3    desertfoxi thi rich nnfirst unleash wuhancoron...
4    bmpiaind step back afteral happen relat wuhanc...
Name: tweet_data, dtype: object

In [151]:
import re 
from textblob import TextBlob 

def clean_tweet(tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
filePointer = open("Sentimetn_Analysis_File.csv","w")
                               
sentiment_data={}  
counter=1
for tweet in data["tweet_data"]:
#     sentiment_data[tweet]=get_tweet_sentiment(tweet)
    data=str(counter)+","+str(tweet)+","+str(get_tweet_sentiment(tweet))+"\n"
    filePointer.write(data)
    counter=counter+1
filePointer.close()


## Question 1: Topic Modeling

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to identify the top 10 topics in the dataset. Before answering this question, please review the materials in lesson 8, especially the code for LDA and LSA. The following information should be reported:

### (1) Features (top n-gram phrases) used for topic modeling.


In [152]:
import gensim
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np


ps = PorterStemmer()

def lemmatize_stemming(text):
    return ps.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result
processed_docs=list()
for i in trainData['tweet']:
    processed_docs.append(preprocess(i))

bigram = gensim.models.Phrases(processed_docs, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[processed_docs], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[processed_docs[0]]])

dictionary = gensim.corpora.Dictionary(processed_docs)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

['bthi', 'rich', 'nnfirst', 'unleash', 'wuhancoronaviru', 'world', 'nnthen', 'suppli', 'substandard', 'medic', 'equip', 'itemxexxa']


### (2) Top 10 clusters for topic modeling.

In [153]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics =10 , 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.060*"wuhancoronaviru" + 0.059*"wuhan" + 0.059*"million" + 0.053*"hunterwolfwang" + 0.053*"hide" + 0.053*"inform" + 0.053*"form" + 0.040*"peopl" + 0.040*"wholxexxa" + 0.020*"starvancouv"


Topic: 1 
Words: 0.033*"case" + 0.033*"million" + 0.033*"markn" + 0.033*"cross" + 0.033*"week" + 0.033*"effort" + 0.033*"corona" + 0.031*"infect" + 0.022*"txexxa" + 0.022*"bblu"


Topic: 2 
Words: 0.053*"wuhancoronaviru" + 0.018*"covid" + 0.018*"spread" + 0.018*"china" + 0.010*"sxexxa" + 0.010*"great" + 0.010*"breptedyoho" + 0.010*"perform" + 0.010*"houseforeigngop" + 0.010*"contain"


Topic: 3 
Words: 0.030*"wuhancoronaviru" + 0.020*"thing" + 0.020*"peopl" + 0.020*"typic" + 0.011*"canadian" + 0.011*"thank" + 0.011*"think" + 0.011*"video" + 0.011*"mask" + 0.011*"social"


Topic: 4 
Words: 0.076*"wuhancoronaviru" + 0.056*"infect" + 0.056*"shabnamhamseda" + 0.056*"close" + 0.056*"ccpchina" + 0.056*"alli" + 0.056*"trudeau" + 0.056*"terrorist" + 0.056*"entirexexxa" + 0.056*"reengin"


T

### (3) Summarize and describe the topic for each cluster.

0 - Virus Effect, describing about the effect of corona on people

1 - Virus Cases, describing the cases per week

2 - China over Virus, describing about how china handled the virus

3 - Political - describing political activities

4 - Terrorism - describing terror activities

5 - Infections - about the infections

6 - Political

7 - Unemployment - about the unemployment issue due to corona

8 - virus Impact - Imapct of virus stages

9 - Corona drug - about the virus drug

### Question 2: Sentiment Analysis
(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: 80% data for training and 20% data for testing.

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively.

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.

In [154]:
import pandas as pd
# train Data
Data = pd.read_csv("Sentimetn_Analysis_File.csv",names=["id","tweet","label"],header=None)

In [155]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
y=Data.label
trainData=Data
x=Data.drop('label',axis=1)
tesData=Data
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['tweet'])
test_vectors = vectorizer.transform(testData['tweet'])

X_train, X_test, y_train, y_test = train_test_split(train_vectors, y, test_size=0.2, random_state=0)

### (2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively.
### (3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [156]:
# Linear Algorithm

import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(X_train, y_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(X_test)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))

print(classification_report(y_test, prediction_linear))
print(accuracy_score(y_test, prediction_linear))

Training time: 0.001994s; Prediction time: 0.000998s
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         2
     neutral       0.75      1.00      0.86         6
    positive       1.00      1.00      1.00         3

    accuracy                           0.82        11
   macro avg       0.58      0.67      0.62        11
weighted avg       0.68      0.82      0.74        11

0.8181818181818182


C:\Users\satya\Anaconda3\envs\tensorflow-sessions\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [157]:
#Random Forest Algorithm

y=Data.label
x=Data.drop('label',axis=1)

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
train_vectors = vectorizer.fit_transform(trainData['tweet'])
test_vectors = vectorizer.transform(testData['tweet'])
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_vectors, y, test_size=0.2, random_state=0)
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)
predictions = text_classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

    negative       1.00      0.50      0.67         2
     neutral       0.86      1.00      0.92         6
    positive       1.00      1.00      1.00         3

    accuracy                           0.91        11
   macro avg       0.95      0.83      0.86        11
weighted avg       0.92      0.91      0.90        11

0.9090909090909091


By comparing both results, both algorithms performed almost same.